In [1]:
#Alquileres temporarios
#Todo lo que no sea hotel, albergues, cabañas, bungalows, hospedajes, hosterías, hostels, residenciales 
#Incluir tipo de establecimiento (habitación, depto, etc), ubicación (dirección) y geolocalización (latitud y longitud).

import geopandas as gpd
import reverse_geocode
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import xlsxwriter
import numpy as np
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
options = Options()
options.add_argument("window-size=950,800") 
options.add_experimental_option("detach", True)
#options.add_argument("--headless")

In [19]:
driver = webdriver.Chrome(executable_path ='C:\\Users\\ITNM\\Downloads\\chromedriver(1).exe', options=options)

C:\Users\ITNM\AppData\Local\Temp\ipykernel_10736\3930261563.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path ='C:\\Users\\ITNM\\Downloads\\chromedriver(1).exe', options=options)


In [3]:
url = 'https://www.airbnb.com.ar/s/Posadas--Misiones--Argentina/homes'
driver.get(url)

In [4]:
buscar = driver.find_element(By.XPATH, '//div[@data-testid="little-search"]').click()

In [5]:
flex = driver.find_element(By.XPATH, '//button[@data-testid="expanded-searchbar-dates-flexible-tab"]').click()

In [6]:
mes = driver.find_element(By.XPATH, '//div[@class="_ko9bki"]').click()

In [7]:
busqueda = driver.find_element(By.XPATH, '//div[@class="s1t4vwjw atm_am_12336oc atm_gz_yjp0fh atm_ll_rdoju8 atm_mk_h2mmj6 atm_wq_qfx8er dir dir-ltr"]').click()

In [8]:
titles = []
desc = []
links = []
precio = []
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}

In [9]:
while True:
    soup = BeautifulSoup(driver.page_source, 'lxml')
    headers = headers

    for x in soup.find_all('div', attrs={'itemprop':"itemListElement"}):
        link = x.find('meta', attrs={'itemprop': "url"})['content']
        if link is None:
            links.append(np.NaN)
        else:
            links.append(link)

        title = x.find('div', attrs={'data-testid':"listing-card-title"})
        if title is None:
            titles.append(np.NaN)
        else:
            titles.append(title.text)

        tipo = x.find('span', attrs={'data-testid':"listing-card-name"})
        if tipo is None:
            desc.append(np.NaN)
        else:
            desc.append(tipo.text)

        px = x.find('span', attrs={'class':"_tyxjp1"})
        if px is None:
            px2 = x.find('span', attrs={'class':"_1y74zjx"})
            precio.append(px2.text)
        else:
            precio.append(px.text)

    #pagination
    driver.execute_script("window.scrollBy(0, 4900);")
    time.sleep(3)

    try:
        next_p = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="Siguiente"]').click()
        time.sleep(3)
    except:
        break 

In [10]:
print(len(titles))

270


In [17]:
links

['www.airbnb.com.ar/rooms/1052244218501072235?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-01-04&check_out=2024-01-09&source_impression_id=p3_1703603222_6r%2FV%2B3tIvEiSDMJT&previous_page_section_name=1000',
 'www.airbnb.com.ar/rooms/668914155741101249?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-01-02&check_out=2024-01-07&source_impression_id=p3_1703603222_G5eaYSZ%2BxbReusaM&previous_page_section_name=1000',
 'www.airbnb.com.ar/rooms/1015111492466981962?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-01-07&check_out=2024-01-12&source_impression_id=p3_1703603222_tx4hTcZJBIDXbpNX&previous_page_section_name=1000',
 'www.airbnb.com.ar/rooms/966664615404633392?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&check_in=2024-01-12&check_out=2024-01-17&source_impression_id=p3_1703603222_xMgU8PTqpSxZFS4W&previous_page_section_name=1000',
 'www.airbnb.com.ar/rooms/104936113894923186

In [22]:
cerrar = driver.find_element(By.XPATH, '//button[@aria-label="Cerrar"]').click()

In [25]:
driver.get('https://www.airbnb.com.ar/rooms/668914155741101249')

In [26]:
p=[]
coments=[]

try:
    pun = driver.find_element(By.XPATH, '//div[@data-testid="pdp-reviews-highlight-banner-host-rating"]')
    p.append(pun.text)
except NoSuchElementException:
    try:
        pun2 = driver.find_element(By.XPATH, '//div[@class="r1lutz1s atm_c8_o7aogt atm_c8_l52nlx__oggzyc dir dir-ltr"]')
        p.append(pun2.text)
    except NoSuchElementException:
        p.append(np.NaN)
        
try:
    coment = driver.find_element(By.XPATH, '//div[@class="r16onr0j atm_c8_exq1xd atm_g3_1pezo5y atm_fr_7aerd4 atm_gq_myb0kj atm_vv_qvpr2i atm_c8_8nb4eg__14195v1 atm_g3_1dpnnv7__14195v1 atm_fr_11dsdeo__14195v1 atm_gq_idpfg4__14195v1 dir dir-ltr"]')
    coments.append(coment.text)
except NoSuchElementException:
    try:
        coment2 = driver.find_element(By.XPATH, '//a[@class="l1ovpqvx atm_1y33qqm_1ggndnn_10saat9 atm_17zvjtw_zk357r_10saat9 atm_w3cb4q_il40rs_10saat9 b1uxatsa atm_c8_1kw7nm4 atm_bx_1kw7nm4 atm_cd_1kw7nm4 atm_ci_1kw7nm4 atm_g3_1kw7nm4 atm_9j_tlke0l_1nos8r_uv4tnr atm_7l_1kw7nm4_pfnrn2 atm_rd_8stvzk_pfnrn2 c1qih7tm atm_1s_glywfm atm_26_1j28jx2 atm_3f_idpfg4 atm_9j_tlke0l atm_gi_idpfg4 atm_l8_idpfg4 atm_vb_1wugsn5 atm_7l_18pqv07 atm_rd_8stvzk atm_5j_mlmjl2 atm_cs_qo5vgd atm_r3_1kw7nm4 atm_mk_h2mmj6 atm_kd_glywfm atm_9j_13gfvf7_1o5j5ji atm_7l_18pqv07_v5whe7 atm_rd_8stvzk_v5whe7 atm_7l_1otlplk_1nos8r_uv4tnr atm_rd_8stvzk_1nos8r_uv4tnr atm_7l_161hw1_4fughm_uv4tnr atm_rd_8stvzk_4fughm_uv4tnr atm_rd_8stvzk_xggcrc_uv4tnr atm_7l_12u4tyr_csw3t1 atm_rd_8stvzk_csw3t1 atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_idpfg4_jo46a5 atm_3f_glywfm_1icshfk atm_kd_glywfm_19774hq atm_7l_18pqv07_1w3cfyq atm_rd_8stvzk_1w3cfyq atm_uc_x37zl0_1w3cfyq atm_70_qporve_1w3cfyq atm_uc_glywfm_1w3cfyq_p88qr9 atm_7l_18pqv07_18zk5v0 atm_rd_8stvzk_18zk5v0 atm_uc_x37zl0_18zk5v0 atm_70_qporve_18zk5v0 atm_uc_glywfm_18zk5v0_p88qr9 atm_7l_161hw1_1o5j5ji atm_rd_8stvzk_1o5j5ji atm_rd_8stvzk_1mj13j2 dir dir-ltr"]')
        coments.append(coment2.text)
    except NoSuchElementException:
        coments.append(np.NaN)


In [27]:
print(p)
print(coments)

['Calificación: 4,97 de 5 estrellas.\n4,97']
['61']


In [28]:
infos=[]
coords = []
p=[]
coments=[]
servicios=[]

for link in links:
    driver.get(f"http://{link}")
    time.sleep(5)
        
    try:
        cerrar = driver.find_element(By.XPATH, '//button[@aria-label="Cerrar"]').click()
    except:
        pass
       
    try:
        info = driver.find_element(By.XPATH, '//ol[@class="lgx66tx atm_gi_idpfg4 atm_l8_idpfg4 dir dir-ltr"]')
        infos.append(info.text)
    except:
        pass
        infos.append(np.NaN)
    
    try:
        pun = driver.find_element(By.XPATH, '//div[@data-testid="pdp-reviews-highlight-banner-host-rating"]')
        p.append(pun.text)
    except NoSuchElementException:
        try:
            pun2 = driver.find_element(By.XPATH, '//div[@class="r1lutz1s atm_c8_o7aogt atm_c8_l52nlx__oggzyc dir dir-ltr"]')
            p.append(pun2.text)
        except NoSuchElementException:
            p.append(np.NaN)
        
    try:
        coment = driver.find_element(By.XPATH, '//div[@class="r16onr0j atm_c8_exq1xd atm_g3_1pezo5y atm_fr_7aerd4 atm_gq_myb0kj atm_vv_qvpr2i atm_c8_8nb4eg__14195v1 atm_g3_1dpnnv7__14195v1 atm_fr_11dsdeo__14195v1 atm_gq_idpfg4__14195v1 dir dir-ltr"]')
        coments.append(coment.text)
    except NoSuchElementException:
        try:
            coment2 = driver.find_element(By.XPATH, '//a[@class="l1ovpqvx atm_1y33qqm_1ggndnn_10saat9 atm_17zvjtw_zk357r_10saat9 atm_w3cb4q_il40rs_10saat9 b1uxatsa atm_c8_1kw7nm4 atm_bx_1kw7nm4 atm_cd_1kw7nm4 atm_ci_1kw7nm4 atm_g3_1kw7nm4 atm_9j_tlke0l_1nos8r_uv4tnr atm_7l_1kw7nm4_pfnrn2 atm_rd_8stvzk_pfnrn2 c1qih7tm atm_1s_glywfm atm_26_1j28jx2 atm_3f_idpfg4 atm_9j_tlke0l atm_gi_idpfg4 atm_l8_idpfg4 atm_vb_1wugsn5 atm_7l_18pqv07 atm_rd_8stvzk atm_5j_mlmjl2 atm_cs_qo5vgd atm_r3_1kw7nm4 atm_mk_h2mmj6 atm_kd_glywfm atm_9j_13gfvf7_1o5j5ji atm_7l_18pqv07_v5whe7 atm_rd_8stvzk_v5whe7 atm_7l_1otlplk_1nos8r_uv4tnr atm_rd_8stvzk_1nos8r_uv4tnr atm_7l_161hw1_4fughm_uv4tnr atm_rd_8stvzk_4fughm_uv4tnr atm_rd_8stvzk_xggcrc_uv4tnr atm_7l_12u4tyr_csw3t1 atm_rd_8stvzk_csw3t1 atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_idpfg4_jo46a5 atm_3f_glywfm_1icshfk atm_kd_glywfm_19774hq atm_7l_18pqv07_1w3cfyq atm_rd_8stvzk_1w3cfyq atm_uc_x37zl0_1w3cfyq atm_70_qporve_1w3cfyq atm_uc_glywfm_1w3cfyq_p88qr9 atm_7l_18pqv07_18zk5v0 atm_rd_8stvzk_18zk5v0 atm_uc_x37zl0_18zk5v0 atm_70_qporve_18zk5v0 atm_uc_glywfm_18zk5v0_p88qr9 atm_7l_161hw1_1o5j5ji atm_rd_8stvzk_1o5j5ji atm_rd_8stvzk_1mj13j2 dir dir-ltr"]')
            coments.append(coment2.text)
        except NoSuchElementException:
            coments.append(np.NaN)
        
    try:
        serv = driver.find_element(By.XPATH, '//h2[contains(text(),"¿Qué ofrece este lugar?")]')
        driver.execute_script("arguments[0].scrollIntoView();", serv)
        listas = driver.find_element(By.XPATH, '//div[@class="c16f2viy atm_9s_1txwivl atm_h_1fhbwtr atm_fc_1y6m0gg atm_be_1g80g66 atm_gz_169eiex atm_h0_169eiex atm_vy_1ku6ftx atm_gz_169eiex__kgj4qw atm_h0_169eiex__kgj4qw atm_vy_1ku6ftx__kgj4qw atm_gz_1cz4k7g__oggzyc atm_h0_1cz4k7g__oggzyc atm_vy_1l2ly1k__oggzyc atm_gz_1cz4k7g__1v156lz atm_h0_1cz4k7g__1v156lz atm_vy_1l2ly1k__1v156lz atm_gz_1cz4k7g__qky54b atm_h0_1cz4k7g__qky54b atm_vy_1l2ly1k__qky54b atm_gz_1cz4k7g__jx8car atm_h0_1cz4k7g__jx8car atm_vy_1l2ly1k__jx8car dir dir-ltr"]')
        servicios.append(listas.text)
        
    except:
        servicios.append(np.NaN)
    
    time.sleep(3)
        
    try:
        maps = driver.find_element(By.XPATH, '//h2[contains(text(),"¿Dónde vas a estar?")]')
        driver.execute_script("arguments[0].scrollIntoView();", maps)
        coordinates = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "//a[contains(@href,'https://maps.google.com/maps?ll=')]")))
        coord_href = str(coordinates.get_attribute('href'))
        parse_coord = re.findall(r"ll=(-?\d+\.\d+),(-?\d+\.\d+)", coord_href)
        parse_coord = parse_coord[0]
        coords.append(parse_coord)
        
    except:
        coords.append(np.NaN)
    

In [29]:
print(len(desc))
print(len(coords))
print(len(p))

270
270
270


In [30]:
print(servicios)

['Cocina\nWifi\nZona de trabajo\nEstacionamiento gratuito en la calle\nTelevisor\nNo disponible: Detector de monóxido de carbono\nDetector de monóxido de carbono\nNo disponible: Detector de humo\nDetector de humo', 'Vistas de la ciudad y el horizonte\nCocina\nWifi\nZona de trabajo\nEstacionamiento gratis en cochera residencial, en las instalaciones: 1 plaza\nNo disponible: Detector de monóxido de carbono\nDetector de monóxido de carbono\nNo disponible: Detector de humo\nDetector de humo', 'Cocina\nWifi\nZona de trabajo\nEstacionamiento gratis en la propiedad\nSe permiten mascotas\nNo disponible: Detector de monóxido de carbono\nDetector de monóxido de carbono\nNo disponible: Detector de humo\nDetector de humo', 'Cocina\nWifi\nPileta De uso compartido: disponible según la temporada\nTelevisión HD de 43 pulgadas con Cable estándar\nAscensor\nNo disponible: Detector de monóxido de carbono\nDetector de monóxido de carbono', 'Cocina\nWifi\nEstacionamiento gratis en la propiedad\nTelevisor\n

In [31]:
df = pd.DataFrame({'Geolocalización':coords,'Precio':precio,'Alojamiento': desc, 'Tipo alojamiento': titles, 'Información':infos, 'Servicios':servicios,'Puntaje':p, 'Comentarios':coments, 'Enlace':links}) 

In [32]:
df

,Geolocalización,Precio,Alojamiento,Tipo alojamiento,Información,Servicios,Puntaje,Comentarios,Enlace
0,"(-27.3741, -55.8885)",$20 USD,Magnífica vista al Río y ubicación estratégica !,Departamento en Posadas,2 huéspedes · 1 dormitorio · 1 baño,Cocina\nWifi\nZona de trabajo\nEstacionamiento...,Novedad,NaN,www.airbnb.com.ar/rooms/1052244218501072235?ad...
1,"(-27.3663, -55.8986)",$25 USD,"Lujoso Departamento Céntrico, Posadas",Departamento en Posadas,4 huéspedes · 1 dormitorio · 1 cama · 1 baño,Vistas de la ciudad y el horizonte\nCocina\nWi...,"Calificación: 4,97 de 5 estrellas.\n4,97",61,www.airbnb.com.ar/rooms/668914155741101249?adu...
2,"(-27.363867, -55.888911)",$25 USD,Amplio Depto. Hermosa Vista,Departamento en Posadas,"4 huéspedes · 1 dormitorio · 1 cama · 1 ,5 baños",Cocina\nWifi\nZona de trabajo\nEstacionamiento...,"Calificación: 5,0 de 5 estrellas.\n5,0",8,www.airbnb.com.ar/rooms/1015111492466981962?ad...
3,"(-27.370091, -55.887163)",$22 USD,Apartamento céntrico en Posadas,Departamento en Posadas,2 huéspedes · 1 dormitorio · 1 cama · 1 baño,Cocina\nWifi\nPileta De uso compartido: dispon...,"Calificación: 4,95 de 5 estrellas.\n4,95",20,www.airbnb.com.ar/rooms/966664615404633392?adu...
4,"(-27.3791, -55.8866)",$24 USD,Alta Vista,Departamento en Posadas,2 huéspedes · 1 dormitorio · 1 cama · 1 baño,Cocina\nWifi\nEstacionamiento gratis en la pro...,Novedad,NaN,www.airbnb.com.ar/rooms/1049361138949231866?ad...
...,...,...,...,...,...,...,...,...,...
265,"(-27.3867, -55.9026)",$17 USD,Moroty 2,"Loft en Posadas, Argentina",2 huéspedes · 1 dormitorio · 1 cama · 1 baño,Cocina\nWifi\nEstacionamiento techado gratis e...,No hay evaluaciones todavía,NaN,www.airbnb.com.ar/rooms/712084235875321666?adu...
266,"(-27.38105, -55.89219)",$37 USD,VICARIO DE APARMENT,"Departamento en Posadas, Argentina",2 huéspedes · 2 dormitorios · 2 camas · 1 baño,Cocina\nWifi\nEstacionamiento gratuito en la c...,"4,78",23 evaluaciones,www.airbnb.com.ar/rooms/20982265?adults=1&chil...
267,"(-27.37203, -55.9268)",$47 USD,Dpto confortable 4 personas,"Departamento en Posadas, Argentina",4 huéspedes · 1 dormitorio · 3 camas · 1 baño,Cocina\nWifi\nEstacionamiento gratis en la pro...,NaN,1 evaluación,www.airbnb.com.ar/rooms/929100352364868678?adu...
268,"(-27.37722, -55.89938)",$17 USD,Depto.: Sweet 07 cerca del Centro.,"Departamento en Posadas, Argentina","3 huéspedes · 1 dormitorio · 1 cama · 1 ,5 baños",Cocina\nWifi\nEstacionamiento gratuito en la c...,"Calificación: 4,95 de 5 estrellas.\n4,95",22,www.airbnb.com.ar/rooms/38701150?adults=1&chil...


In [33]:
df.index = np.arange(1, len(df) + 1)
#df = df.replace(r'\n',' ', regex=True)
df['Tipo alojamiento'] = df['Tipo alojamiento'].str.strip()
df['Precio'] = df['Precio'].str.strip()
#df['Puntaje']=df['Puntaje'].str.strip('·')
#df['Servicios']=df['Servicios'].str.strip('[')
#df['Servicios']=df['Servicios'].str.strip(']')
df['Servicios']=df['Servicios'].replace("\n", ", ", regex=True)

In [34]:
df

,Geolocalización,Precio,Alojamiento,Tipo alojamiento,Información,Servicios,Puntaje,Comentarios,Enlace
1,"(-27.3741, -55.8885)",$20 USD,Magnífica vista al Río y ubicación estratégica !,Departamento en Posadas,2 huéspedes · 1 dormitorio · 1 baño,"Cocina, Wifi, Zona de trabajo, Estacionamiento...",Novedad,NaN,www.airbnb.com.ar/rooms/1052244218501072235?ad...
2,"(-27.3663, -55.8986)",$25 USD,"Lujoso Departamento Céntrico, Posadas",Departamento en Posadas,4 huéspedes · 1 dormitorio · 1 cama · 1 baño,"Vistas de la ciudad y el horizonte, Cocina, Wi...","Calificación: 4,97 de 5 estrellas.\n4,97",61,www.airbnb.com.ar/rooms/668914155741101249?adu...
3,"(-27.363867, -55.888911)",$25 USD,Amplio Depto. Hermosa Vista,Departamento en Posadas,"4 huéspedes · 1 dormitorio · 1 cama · 1 ,5 baños","Cocina, Wifi, Zona de trabajo, Estacionamiento...","Calificación: 5,0 de 5 estrellas.\n5,0",8,www.airbnb.com.ar/rooms/1015111492466981962?ad...
4,"(-27.370091, -55.887163)",$22 USD,Apartamento céntrico en Posadas,Departamento en Posadas,2 huéspedes · 1 dormitorio · 1 cama · 1 baño,"Cocina, Wifi, Pileta De uso compartido: dispon...","Calificación: 4,95 de 5 estrellas.\n4,95",20,www.airbnb.com.ar/rooms/966664615404633392?adu...
5,"(-27.3791, -55.8866)",$24 USD,Alta Vista,Departamento en Posadas,2 huéspedes · 1 dormitorio · 1 cama · 1 baño,"Cocina, Wifi, Estacionamiento gratis en la pro...",Novedad,NaN,www.airbnb.com.ar/rooms/1049361138949231866?ad...
...,...,...,...,...,...,...,...,...,...
266,"(-27.3867, -55.9026)",$17 USD,Moroty 2,"Loft en Posadas, Argentina",2 huéspedes · 1 dormitorio · 1 cama · 1 baño,"Cocina, Wifi, Estacionamiento techado gratis e...",No hay evaluaciones todavía,NaN,www.airbnb.com.ar/rooms/712084235875321666?adu...
267,"(-27.38105, -55.89219)",$37 USD,VICARIO DE APARMENT,"Departamento en Posadas, Argentina",2 huéspedes · 2 dormitorios · 2 camas · 1 baño,"Cocina, Wifi, Estacionamiento gratuito en la c...","4,78",23 evaluaciones,www.airbnb.com.ar/rooms/20982265?adults=1&chil...
268,"(-27.37203, -55.9268)",$47 USD,Dpto confortable 4 personas,"Departamento en Posadas, Argentina",4 huéspedes · 1 dormitorio · 3 camas · 1 baño,"Cocina, Wifi, Estacionamiento gratis en la pro...",NaN,1 evaluación,www.airbnb.com.ar/rooms/929100352364868678?adu...
269,"(-27.37722, -55.89938)",$17 USD,Depto.: Sweet 07 cerca del Centro.,"Departamento en Posadas, Argentina","3 huéspedes · 1 dormitorio · 1 cama · 1 ,5 baños","Cocina, Wifi, Estacionamiento gratuito en la c...","Calificación: 4,95 de 5 estrellas.\n4,95",22,www.airbnb.com.ar/rooms/38701150?adults=1&chil...


In [35]:
df['Sitio web'] = 'AirBnB'
df['Fecha consulta'] = pd.Timestamp.today().strftime('%d/%m/%Y')

In [36]:
df['Enlace'] = 'https://' + df['Enlace'].astype(str)

In [37]:
df[['Capacidad','Dormitorios','Camas','Baños']] = df['Información'].str.split('·',expand=True)

In [38]:
df

,Geolocalización,Precio,Alojamiento,Tipo alojamiento,Información,Servicios,Puntaje,Comentarios,Enlace,Sitio web,Fecha consulta,Capacidad,Dormitorios,Camas,Baños
1,"(-27.3741, -55.8885)",$20 USD,Magnífica vista al Río y ubicación estratégica !,Departamento en Posadas,2 huéspedes · 1 dormitorio · 1 baño,"Cocina, Wifi, Zona de trabajo, Estacionamiento...",Novedad,NaN,https://www.airbnb.com.ar/rooms/10522442185010...,AirBnB,26/12/2023,2 huéspedes,1 dormitorio,1 baño,None
2,"(-27.3663, -55.8986)",$25 USD,"Lujoso Departamento Céntrico, Posadas",Departamento en Posadas,4 huéspedes · 1 dormitorio · 1 cama · 1 baño,"Vistas de la ciudad y el horizonte, Cocina, Wi...","Calificación: 4,97 de 5 estrellas.\n4,97",61,https://www.airbnb.com.ar/rooms/66891415574110...,AirBnB,26/12/2023,4 huéspedes,1 dormitorio,1 cama,1 baño
3,"(-27.363867, -55.888911)",$25 USD,Amplio Depto. Hermosa Vista,Departamento en Posadas,"4 huéspedes · 1 dormitorio · 1 cama · 1 ,5 baños","Cocina, Wifi, Zona de trabajo, Estacionamiento...","Calificación: 5,0 de 5 estrellas.\n5,0",8,https://www.airbnb.com.ar/rooms/10151114924669...,AirBnB,26/12/2023,4 huéspedes,1 dormitorio,1 cama,"1 ,5 baños"
4,"(-27.370091, -55.887163)",$22 USD,Apartamento céntrico en Posadas,Departamento en Posadas,2 huéspedes · 1 dormitorio · 1 cama · 1 baño,"Cocina, Wifi, Pileta De uso compartido: dispon...","Calificación: 4,95 de 5 estrellas.\n4,95",20,https://www.airbnb.com.ar/rooms/96666461540463...,AirBnB,26/12/2023,2 huéspedes,1 dormitorio,1 cama,1 baño
5,"(-27.3791, -55.8866)",$24 USD,Alta Vista,Departamento en Posadas,2 huéspedes · 1 dormitorio · 1 cama · 1 baño,"Cocina, Wifi, Estacionamiento gratis en la pro...",Novedad,NaN,https://www.airbnb.com.ar/rooms/10493611389492...,AirBnB,26/12/2023,2 huéspedes,1 dormitorio,1 cama,1 baño
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,"(-27.3867, -55.9026)",$17 USD,Moroty 2,"Loft en Posadas, Argentina",2 huéspedes · 1 dormitorio · 1 cama · 1 baño,"Cocina, Wifi, Estacionamiento techado gratis e...",No hay evaluaciones todavía,NaN,https://www.airbnb.com.ar/rooms/71208423587532...,AirBnB,26/12/2023,2 huéspedes,1 dormitorio,1 cama,1 baño
267,"(-27.38105, -55.89219)",$37 USD,VICARIO DE APARMENT,"Departamento en Posadas, Argentina",2 huéspedes · 2 dormitorios · 2 camas · 1 baño,"Cocina, Wifi, Estacionamiento gratuito en la c...","4,78",23 evaluaciones,https://www.airbnb.com.ar/rooms/20982265?adult...,AirBnB,26/12/2023,2 huéspedes,2 dormitorios,2 camas,1 baño
268,"(-27.37203, -55.9268)",$47 USD,Dpto confortable 4 personas,"Departamento en Posadas, Argentina",4 huéspedes · 1 dormitorio · 3 camas · 1 baño,"Cocina, Wifi, Estacionamiento gratis en la pro...",NaN,1 evaluación,https://www.airbnb.com.ar/rooms/92910035236486...,AirBnB,26/12/2023,4 huéspedes,1 dormitorio,3 camas,1 baño
269,"(-27.37722, -55.89938)",$17 USD,Depto.: Sweet 07 cerca del Centro.,"Departamento en Posadas, Argentina","3 huéspedes · 1 dormitorio · 1 cama · 1 ,5 baños","Cocina, Wifi, Estacionamiento gratuito en la c...","Calificación: 4,95 de 5 estrellas.\n4,95",22,https://www.airbnb.com.ar/rooms/38701150?adult...,AirBnB,26/12/2023,3 huéspedes,1 dormitorio,1 cama,"1 ,5 baños"


In [72]:
del df['Información']
df

,Geolocalización,Precio,Alojamiento,Tipo alojamiento,Servicios,Puntaje,Comentarios,Enlace,Sitio web,Fecha consulta,Capacidad,Dormitorios,Camas,Baños
1,"(-27.38728, -55.89285)",$20 USD,El Portal - Departamento Posadas,"Departamento en Posadas, Argentina","Cocina, Wifi, Estacionamiento gratuito en la c...","Calificación: 4,98 de 5 estrellas.\n4,98",57,https://www.airbnb.com.ar/rooms/87321207237994...,AirBnB,19/12/2023,3 huéspedes,1 dormitorio,3 camas,1 baño
2,"(-27.37703, -55.90481)",$19 USD,"Depto. con patio, céntrico, luminoso, amplio.","Departamento en Posadas, Argentina","Cocina, Wifi, Zona de trabajo, Estacionamiento...","Calificación: 4,95 de 5 estrellas.\n4,95",40,https://www.airbnb.com.ar/rooms/41959923?adult...,AirBnB,19/12/2023,4 huéspedes,1 dormitorio,2 camas,1 baño
3,"(-27.3791, -55.8866)",$68 USD,Dpto moderno con vista al río y a la ciudad,Departamento en Posadas,"Vistas de la ciudad y el horizonte, Vista al r...",Novedad,2 evaluaciones,https://www.airbnb.com.ar/rooms/10340013428952...,AirBnB,19/12/2023,4 huéspedes,2 dormitorios,3 camas,2 baños
4,"(-27.3698, -55.8987)",$23 USD,Departamento Céntrico 2,Departamento en Posadas,"Cocina, Wifi, Se permiten mascotas, Televisor,...","Calificación: 4,94 de 5 estrellas.\n4,94",17,https://www.airbnb.com.ar/rooms/89622977988509...,AirBnB,19/12/2023,3 huéspedes,1 dormitorio,2 camas,1 baño
5,"(-27.3676, -55.8937)",$37 USD,Departamento céntrico en Posadas,Departamento en Posadas,"Cocina, Wifi, Pileta, Televisor, Lavarropas, N...",Novedad,1 evaluación,https://www.airbnb.com.ar/rooms/10318659952184...,AirBnB,19/12/2023,3 huéspedes,1 dormitorio,2 camas,"1 ,5 baños"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,"(-27.33437, -55.86387)",$34 USD,Encantador hotel con piscina y estacionamiento,"Hotel en Encarnación, Paraguay","Wifi, Estacionamiento gratis en la propiedad, ...","4,57",7 evaluaciones,https://www.airbnb.com.ar/rooms/54142185?adult...,AirBnB,19/12/2023,3 huéspedes,1 dormitorio,2 camas,1 baño privado
267,"(-27.36851, -55.88678)",$71 USD,Espectacular Departamento!,"Departamento en Posadas, Argentina","Cocina, Wifi, Televisor, Aire acondicionado, S...",NaN,2 evaluaciones,https://www.airbnb.com.ar/rooms/89049377562215...,AirBnB,19/12/2023,4 huéspedes,1 dormitorio,2 camas,1 baño
268,"(-27.42583, -55.92231)",$7 USD,Habitación Amarilla en Palmeira Bonita - Posadas,"Habitación en Posadas, Argentina","Cerradura en la puerta del dormitorio, Cocina,...",No hay evaluaciones todavía,Mostrar más,https://www.airbnb.com.ar/rooms/41749209?adult...,AirBnB,19/12/2023,1 cama doble,Baño compartido,None,None
269,"(-27.3762, -55.8975)",$26 USD,Departamento temporario amoblado Posadas - 6 D.,"Departamento en Posadas, Argentina","Vistas de la ciudad y el horizonte, Vista al r...",NaN,1 evaluación,https://www.airbnb.com.ar/rooms/49142499?adult...,AirBnB,19/12/2023,2 huéspedes,1 dormitorio,1 cama,1 baño


In [39]:
df.to_excel('AirBnB_Posadas_26-12-23.xlsx')